In [1]:
from hloc.pipelines.Cambridge.utils import evaluate
from pathlib import Path
from IPython.display import display
import pandas as pd

In [2]:
d = {}

for suffix in ['intra_2015', '2015', '2015_2016', '2015_2016_2018']:
    d[suffix] = {}
    for res in ['results.txt', 'results_covis_oracle.txt']:
        gt_dir = Path('datasets/toureiffel') / suffix
        model = gt_dir / 'empty_all'
        list_query = gt_dir / 'list_query.txt'

        results = Path('outputs') / 'TourEiffel' / suffix / res

        d[suffix][res] = evaluate(model, results, list_query, ext='.txt')

[2022/04/27 15:12:40 hloc.pipelines.Cambridge.utils INFO] Results for file results.txt:
Median errors: 0.001m, 0.021deg
Percentage of test images localized within:
	1cm, 1deg : 97.95%
	2cm, 2deg : 99.18%
	3cm, 3deg : 99.59%
	5cm, 5deg : 99.90%
	25cm, 2deg : 100.00%
	50cm, 5deg : 100.00%
	500cm, 10deg : 100.00%
[2022/04/27 15:12:40 hloc.pipelines.Cambridge.utils INFO] Results for file results_covis_oracle.txt:
Median errors: 0.001m, 0.021deg
Percentage of test images localized within:
	1cm, 1deg : 98.46%
	2cm, 2deg : 99.38%
	3cm, 3deg : 99.69%
	5cm, 5deg : 99.90%
	25cm, 2deg : 100.00%
	50cm, 5deg : 100.00%
	500cm, 10deg : 100.00%
[2022/04/27 15:12:41 hloc.pipelines.Cambridge.utils INFO] Results for file results.txt:
Median errors: 0.401m, 4.670deg
Percentage of test images localized within:
	1cm, 1deg : 11.24%
	2cm, 2deg : 22.16%
	3cm, 3deg : 29.78%
	5cm, 5deg : 36.94%
	25cm, 2deg : 46.04%
	50cm, 5deg : 49.58%
	500cm, 10deg : 51.69%
[2022/04/27 15:12:41 hloc.pipelines.Cambridge.utils IN

In [3]:
suffix_map = {
    'intra_2015': '2015',
    '2015': '2016',
    '2015_2016': '2018',
    '2015_2016_2018': '2020'
}

dfs = []
for suffix, data in d.items():
    df = pd.DataFrame.from_dict(data, orient='index')
    df.insert(0, 'Global retrieval', df.index.map(lambda x: 'NetVLAD' if x == 'results.txt' else 'Covisibility'))
    df.index = [suffix_map[suffix]] * len(df.index)
    dfs.append(df)
df = pd.concat(dfs)

df['Median translation'] = df['Median translation'].map(lambda x: f'{x:.3f}m')
df['Median rotation'] = df['Median rotation'].map(lambda x: f'{x:.3f}°')
for col in df.columns[3:]:
    df[col] = df[col].map(lambda x: f'{100*x:.2f}%')

In [4]:
df

,Global retrieval,Median translation,Median rotation,"1cm, 1°","2cm, 2°","3cm, 3°","5cm, 5°","25cm, 2°","50cm, 5°","500cm, 10°"
2015,NetVLAD,0.001m,0.021°,97.95%,99.18%,99.59%,99.90%,100.00%,100.00%,100.00%
2015,Covisibility,0.001m,0.021°,98.46%,99.38%,99.69%,99.90%,100.00%,100.00%,100.00%
2016,NetVLAD,0.401m,4.670°,11.24%,22.16%,29.78%,36.94%,46.04%,49.58%,51.69%
2016,Covisibility,0.024m,0.320°,23.89%,45.20%,56.53%,66.58%,77.55%,82.03%,84.76%
2018,NetVLAD,0.623m,5.639°,9.45%,19.70%,26.31%,34.16%,44.48%,48.45%,51.56%
2018,Covisibility,0.022m,0.273°,23.12%,47.20%,60.95%,71.93%,88.51%,93.23%,95.36%
2020,NetVLAD,1.281m,11.437°,4.33%,11.66%,18.05%,26.32%,40.58%,45.54%,49.13%
2020,Covisibility,0.020m,0.237°,21.18%,49.03%,64.66%,79.91%,94.73%,96.97%,97.73%
